In [1]:
import math

## Setup

In [2]:
# original (1000Mbps)
model_size = 11544930 * 4 * 8
param_size = round(model_size / 32)
add_p = 0.0014
add_time_per_million = 0.012
mul_time_per_million = 0

nr_layer = 5
nr_worker = 3
recv_ratio = 91.55/100

bandwidth_dict = {}
bandwidth_dict["s2c"] = 47.2 * 1000 * 1000
bandwidth_dict["c2c"] = 48.97 * 1000 * 1000
bandwidth_dict["c2w"] = 465 * 1000 * 1000

time_cmd = 0.002

gis_real_value = 8.66
emp_real_value = 58
agg_real_value = 0.23
dec_real_value = 5.2

In [3]:
# # 500Mbps
# model_size = 11544930 * 4 * 8
# param_size = round(model_size / 32)
# add_p = 0.0014
# add_time_per_million = 0.012
# mul_time_per_million = 0

# nr_layer = 5
# nr_worker = 3
# recv_ratio = 88/100
# # recv_ratio = (100-12.3)/100

# bandwidth_dict = {}
# bandwidth_dict["s2c"] = 43.85 * 1000 * 1000
# bandwidth_dict["c2c"] = 46.42 * 1000 * 1000
# bandwidth_dict["c2w"] = 313.92 * 1000 * 1000

# time_cmd = 0.002

# gis_real_value = 9.72
# emp_real_value = 61.99
# agg_real_value = 0.23
# dec_real_value = 7.42

In [4]:
# # 100Mbps
# model_size = 11544930 * 4 * 8
# param_size = round(model_size / 32)
# add_p = 0.0014
# add_time_per_million = 0.012
# mul_time_per_million = 0

# nr_layer = 5
# nr_worker = 3
# recv_ratio = 63.38/100
# # recv_ratio = (100-37.3)/100

# bandwidth_dict = {}
# # bandwidth_dict["s2c"] = 33.38 * 1000 * 1000
# bandwidth_dict["s2c"] = 33.28 * 1000 * 1000
# bandwidth_dict["c2c"] = 34.1 * 1000 * 1000
# bandwidth_dict["c2w"] = 86.41 * 1000 * 1000

# time_cmd = 0.002

# gis_real_value = 18.35
# emp_real_value = 76.94
# agg_real_value = 0.24
# dec_real_value = 25.9

In [5]:
# # 10Mbps
# model_size = 11544930 * 4 * 8
# param_size = round(model_size / 32)
# add_p = 0.0014
# add_time_per_million = 0.012
# mul_time_per_million = 0

# nr_layer = 5
# nr_worker = 3
# recv_ratio = (100-85)/100

# bandwidth_dict = {}
# bandwidth_dict["s2c"] = 8.07 * 1000 * 1000
# bandwidth_dict["c2c"] = 8.1 * 1000 * 1000
# bandwidth_dict["c2w"] = 9.46 * 1000 * 1000

# time_cmd = 0.002

# gis_real_value = 122.5
# emp_real_value = 222.5
# agg_real_value = 0.24
# dec_real_value = 234.6

In [6]:
def ratio(pred_value, real_value):
    return pred_value / real_value

In [7]:
recv_ratio

0.9155

## GIS Prediction

In [8]:
def gis_prediction():
    return model_size / bandwidth_dict["s2c"] + model_size / bandwidth_dict["s2c"] * (1-recv_ratio) * (nr_worker-1)

In [9]:
gis_prediction()

9.149846216949154

In [10]:
gis_real_value

8.66

In [11]:
ratio(gis_prediction(), gis_real_value)

1.0565642282851218

## Local Training Prediction

## EMP

In [12]:
def emp_prediction():
    time_fixed_model = (2 * model_size) / bandwidth_dict["c2c"]
    return (nr_worker - 1) * (1 + recv_ratio) * time_fixed_model + mul_time_per_million * (param_size/1000000)

In [13]:
emp_prediction()

57.80339222217684

In [14]:
emp_real_value

58

In [15]:
ratio(emp_prediction(), emp_real_value)

0.9966102107271869

## Aggregation prediction

In [16]:
add_p * (7080000 / 1000000)

0.009912

In [17]:
time_cmd

0.002

In [18]:
p_list = [500, 20, 25000, 50, 7080000, 8850, 4425000, 500, 5000, 10]
agg_time = 0
for p in p_list:
    single_add_time = add_p * (p / 1000000) + time_cmd * nr_worker
    layer_time = single_add_time * (nr_worker-1)
    print(single_add_time, layer_time)
    agg_time += layer_time

0.0060007 0.0120014
0.006000028 0.012000056
0.006035 0.01207
0.00600007 0.01200014
0.015912000000000003 0.031824000000000005
0.00601239 0.01202478
0.012195000000000001 0.024390000000000002
0.0060007 0.0120014
0.006007 0.012014
0.006000014 0.012000028


In [19]:
add_p

0.0014

In [20]:
def agg_prediction():
    return 2 * nr_layer * ((nr_worker - 1) * (nr_worker * time_cmd)) + (nr_worker - 1) * nr_worker *  (param_size/1000000) * add_p

In [21]:
# def agg_prediction():
#     return 2 * nr_layer * ((nr_worker - 1) * (nr_worker * time_cmd)) + (nr_worker - 1) *  (param_size/1000000) * add_p

In [22]:
# def agg_prediction():
#     return 2 * nr_layer * ((nr_worker - 1) * nr_worker * time_cmd) + (param_size/1000000) * add_time_per_million

In [23]:
agg_prediction()

0.21697741199999998

In [24]:
ratio(agg_prediction(), agg_real_value)

0.943380052173913

## Decryption prediction

In [25]:
def dec_prediction():
    return nr_worker * (model_size * 2 / bandwidth_dict["c2w"])

In [26]:
dec_prediction()

4.766938838709677

In [29]:
dec_real_value

5.2

In [28]:
ratio(dec_prediction(), dec_real_value)

0.9167190074441687